# Random Forest + One Hot Encoder
https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd

le meilleur pour le moment

In [1487]:
import pandas as pd
import numpy as np

# import du csv
data = pd.read_csv('csv_correction.csv')
data

,longitude,latitude,clc_quartier,clc_secteur,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,fk_stadedev,fk_port,fk_pied,fk_situation,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,fk_nomtech,villeca,feuillage,remarquable
0,3.293264,49.840500,Quartier du Centre-Ville,Quai Gayant,6.0,2.0,37.0,EN PLACE,Jeune,semi libre,gazon,Alignement,Non,15.0,5.0,0.0,QUERUB,VILLE,Feuillu,Non
1,3.273380,49.861409,Quartier du Vermandois,Stade Cepy,13.0,1.0,160.0,EN PLACE,Adulte,semi libre,gazon,Groupe,Non,50.0,10.0,0.0,PINNIGnig,VILLE,Conifère,Non
2,3.289068,49.844513,Quartier du Centre-Ville,Rue Villebois Mareuil,12.0,3.0,116.0,REMPLACÉ,Adulte,semi libre,gazon,Alignement,Non,30.0,10.0,0.0,ACEPSE,VILLE,Feuillu,Non
3,3.302387,49.861778,Quartier de l'Europe,Square des Marronniers,16.0,3.0,150.0,EN PLACE,Adulte,semi libre,gazon,Groupe,Non,50.0,2.0,0.0,ACEPLA,VILLE,Feuillu,Non
4,3.304047,49.858446,Quartier de l'Europe,Avenue Buffon,5.0,2.0,170.0,Essouché,Adulte,réduit,gazon,Isolé,Non,40.0,2.0,0.0,SALBAB,VILLE,Feuillu,Non
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7404,3.283505,49.848695,Quartier du Centre-Ville,Place Edouard Branly,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Isolé,Non,5.0,1.0,0.0,ALBJUL,VILLE,Feuillu,Non
7405,3.283436,49.848682,Quartier du Centre-Ville,Place Edouard Branly,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Isolé,Non,3.0,1.0,0.0,ALBJULcho,VILLE,Feuillu,Non
7406,3.297078,49.860413,Quartier de l'Europe,rue Hertz,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Isolé,Non,5.0,1.0,0.0,CEDLIB,VILLE,Conifère,Non
7407,3.291685,49.848009,Quartier du Centre-Ville,Square rue Fréreuse,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Alignement,Non,5.0,1.0,0.0,FAGSYLdawgol,VILLE,Feuillu,Non


In [1488]:
data.columns

Index(['longitude', 'latitude', 'clc_quartier', 'clc_secteur', 'haut_tot',
       'haut_tronc', 'tronc_diam', 'fk_arb_etat', 'fk_stadedev', 'fk_port',
       'fk_pied', 'fk_situation', 'fk_revetement', 'age_estim',
       'fk_prec_estim', 'clc_nbr_diag', 'fk_nomtech', 'villeca', 'feuillage',
       'remarquable'],
      dtype='object')

In [1489]:
from sklearn import preprocessing, feature_selection, model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import cross_val_predict, cross_val_score
import time
from imblearn.over_sampling import SMOTE


In [1490]:
indexDrop = data[((data['fk_arb_etat'] == 'ABATTU') | (data['fk_arb_etat'] == 'EN PLACE') | (data['fk_arb_etat'] =='REMPLACÉ') | (data['fk_arb_etat'] == 'SUPPRIMÉ') )].index
indexDrop

Index([   0,    1,    2,    3,    5,    6,    7,    8,    9,   10,
       ...
       7399, 7400, 7401, 7402, 7403, 7404, 7405, 7406, 7407, 7408],
      dtype='int64', length=7217)

In [1491]:
data.drop(indexDrop, inplace = True)

In [1492]:
data

,longitude,latitude,clc_quartier,clc_secteur,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,fk_stadedev,fk_port,fk_pied,fk_situation,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,fk_nomtech,villeca,feuillage,remarquable
4,3.304047,49.858446,Quartier de l'Europe,Avenue Buffon,5.0,2.0,170.0,Essouché,Adulte,réduit,gazon,Isolé,Non,40.0,2.0,0.0,SALBAB,VILLE,Feuillu,Non
73,3.270746,49.844440,Quartier Saint-Martin - Oëstres,Rue Jean Mermoz,12.0,3.0,160.0,Essouché,Adulte,réduit relâché,gazon,Isolé,Non,50.0,10.0,1.0,PLAACE,VILLE,Feuillu,Non
76,3.270951,49.844588,Quartier Saint-Martin - Oëstres,Rue Jean Mermoz,13.0,4.0,155.0,Essouché,Adulte,réduit relâché,gazon,Isolé,Non,50.0,10.0,2.0,PLAACE,VILLE,Feuillu,Non
92,3.294703,49.849927,Quartier Remicourt,Parc des Champs-Elysées,24.0,9.0,245.0,Essouché,Adulte,semi libre,gazon,Groupe,Non,80.0,20.0,2.0,ACEPLA,VILLE,Feuillu,Non
104,3.294280,49.849615,Quartier Remicourt,Parc des Champs-Elysées,20.0,14.0,180.0,Essouché,Adulte,semi libre,gazon,Groupe,Non,80.0,20.0,2.0,TILCOR,VILLE,Feuillu,Non
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6886,3.296399,49.848276,Quartier Remicourt,Place du Nain d'Alsace,9.0,3.0,85.0,Essouché,Jeune,architecturé,Terre,Isolé,Non,25.0,5.0,0.0,BETPEN,VILLE,Feuillu,Non
6887,3.296366,49.848250,Quartier Remicourt,Place du Nain d'Alsace,9.0,3.0,86.0,Essouché,Jeune,architecturé,Terre,Isolé,Non,25.0,5.0,0.0,BETPEN,VILLE,Feuillu,Non
6888,3.296433,49.848229,Quartier Remicourt,Place du Nain d'Alsace,9.0,4.0,98.0,Essouché,Jeune,architecturé,Terre,Isolé,Non,25.0,5.0,0.0,BETPEN,VILLE,Feuillu,Non
6977,3.265997,49.836176,Quartier Saint-Martin - Oëstres,Ecole maternelle Ernest Lavisse,5.0,2.0,25.0,Non essouché,Jeune,Libre,Terre,Isolé,Non,10.0,2.0,0.0,FAGSYLpur,VILLE,Feuillu,Non


In [1493]:
data["fk_arb_etat"].value_counts()

fk_arb_etat
Essouché        165
Non essouché     27
Name: count, dtype: int64

passage de donnees str en int

In [1494]:
from sklearn.preprocessing import OneHotEncoder

In [1495]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')

In [1496]:
data.columns

Index(['longitude', 'latitude', 'clc_quartier', 'clc_secteur', 'haut_tot',
       'haut_tronc', 'tronc_diam', 'fk_arb_etat', 'fk_stadedev', 'fk_port',
       'fk_pied', 'fk_situation', 'fk_revetement', 'age_estim',
       'fk_prec_estim', 'clc_nbr_diag', 'fk_nomtech', 'villeca', 'feuillage',
       'remarquable'],
      dtype='object')

In [1497]:
#categorical_data_cols = ['fk_stadedev', 'clc_quartier', 'clc_secteur', 'feuillage', 'fk_port', 'fk_situation', 'fk_nomtech']
categorical_data_cols = ['clc_quartier', 'clc_secteur', 'fk_stadedev', 'fk_port', 'fk_pied', 'fk_situation',  'fk_nomtech', 'villeca', 'feuillage']
boolean_cols = ['fk_revetement', 'remarquable']

On encode les donnees categorielles et supprimme les colonnes originales.

In [1498]:

# 'fk_arb_etat_bool',
dict_encoders = {}
for col in categorical_data_cols:
  ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')
  ohe.fit(data[[col]])
  dict_encoders[col] = ohe
  print(type(dict_encoders[col]))
  print(col)
  ohetransform = ohe.transform(data[[col]])
  data = data.join(ohetransform)

data.drop(categorical_data_cols, axis = 1, inplace = True) 
data
  #print(encoder.categories_)
  #data[col] = encoded

<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
clc_quartier
<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
clc_secteur
<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
fk_stadedev
<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
fk_port
<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
fk_pied
<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
fk_situation
<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
fk_nomtech
<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
villeca
<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
feuillage


,longitude,latitude,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,...,fk_nomtech_ROBPSE,fk_nomtech_SALBAB,fk_nomtech_SORAUC,fk_nomtech_TAXDIS,fk_nomtech_TILCOR,fk_nomtech_TILTOM,villeca_CASQ,villeca_VILLE,feuillage_Conifère,feuillage_Feuillu
4,3.304047,49.858446,5.0,2.0,170.0,Essouché,Non,40.0,2.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
73,3.270746,49.844440,12.0,3.0,160.0,Essouché,Non,50.0,10.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
76,3.270951,49.844588,13.0,4.0,155.0,Essouché,Non,50.0,10.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
92,3.294703,49.849927,24.0,9.0,245.0,Essouché,Non,80.0,20.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
104,3.294280,49.849615,20.0,14.0,180.0,Essouché,Non,80.0,20.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6886,3.296399,49.848276,9.0,3.0,85.0,Essouché,Non,25.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
6887,3.296366,49.848250,9.0,3.0,86.0,Essouché,Non,25.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
6888,3.296433,49.848229,9.0,4.0,98.0,Essouché,Non,25.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
6977,3.265997,49.836176,5.0,2.0,25.0,Non essouché,Non,10.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [1499]:
dict_encoders

{'clc_quartier': OneHotEncoder(handle_unknown='ignore', sparse_output=False),
 'clc_secteur': OneHotEncoder(handle_unknown='ignore', sparse_output=False),
 'fk_stadedev': OneHotEncoder(handle_unknown='ignore', sparse_output=False),
 'fk_port': OneHotEncoder(handle_unknown='ignore', sparse_output=False),
 'fk_pied': OneHotEncoder(handle_unknown='ignore', sparse_output=False),
 'fk_situation': OneHotEncoder(handle_unknown='ignore', sparse_output=False),
 'fk_nomtech': OneHotEncoder(handle_unknown='ignore', sparse_output=False),
 'villeca': OneHotEncoder(handle_unknown='ignore', sparse_output=False),
 'feuillage': OneHotEncoder(handle_unknown='ignore', sparse_output=False)}

Pareil pour les colonnes booléennes Oui/Non

In [1500]:
for col in boolean_cols:
  new_col = []
  for cell in data[col]:
    new_col.append(1 if cell=="Oui" else 0)
  print(col)
  print(new_col)
  data[col] = new_col

fk_revetement
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
remarquable
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [1501]:
data

,longitude,latitude,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,...,fk_nomtech_ROBPSE,fk_nomtech_SALBAB,fk_nomtech_SORAUC,fk_nomtech_TAXDIS,fk_nomtech_TILCOR,fk_nomtech_TILTOM,villeca_CASQ,villeca_VILLE,feuillage_Conifère,feuillage_Feuillu
4,3.304047,49.858446,5.0,2.0,170.0,Essouché,0,40.0,2.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
73,3.270746,49.844440,12.0,3.0,160.0,Essouché,0,50.0,10.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
76,3.270951,49.844588,13.0,4.0,155.0,Essouché,0,50.0,10.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
92,3.294703,49.849927,24.0,9.0,245.0,Essouché,0,80.0,20.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
104,3.294280,49.849615,20.0,14.0,180.0,Essouché,0,80.0,20.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6886,3.296399,49.848276,9.0,3.0,85.0,Essouché,0,25.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
6887,3.296366,49.848250,9.0,3.0,86.0,Essouché,0,25.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
6888,3.296433,49.848229,9.0,4.0,98.0,Essouché,0,25.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
6977,3.265997,49.836176,5.0,2.0,25.0,Non essouché,0,10.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


Les colonnes qui vont etre utilisees dans le modele : on prend TOUT

In [1502]:
#all_cols_list = ["haut_tot", 'haut_tronc', 'tronc_diam', 'fk_stadedev', 'age_estim', 'clc_quartier', 'clc_secteur', 'feuillage', 'fk_port', 'fk_situation', 'fk_nomtech']

In [1503]:
data

,longitude,latitude,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,...,fk_nomtech_ROBPSE,fk_nomtech_SALBAB,fk_nomtech_SORAUC,fk_nomtech_TAXDIS,fk_nomtech_TILCOR,fk_nomtech_TILTOM,villeca_CASQ,villeca_VILLE,feuillage_Conifère,feuillage_Feuillu
4,3.304047,49.858446,5.0,2.0,170.0,Essouché,0,40.0,2.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
73,3.270746,49.844440,12.0,3.0,160.0,Essouché,0,50.0,10.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
76,3.270951,49.844588,13.0,4.0,155.0,Essouché,0,50.0,10.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
92,3.294703,49.849927,24.0,9.0,245.0,Essouché,0,80.0,20.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
104,3.294280,49.849615,20.0,14.0,180.0,Essouché,0,80.0,20.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6886,3.296399,49.848276,9.0,3.0,85.0,Essouché,0,25.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
6887,3.296366,49.848250,9.0,3.0,86.0,Essouché,0,25.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
6888,3.296433,49.848229,9.0,4.0,98.0,Essouché,0,25.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
6977,3.265997,49.836176,5.0,2.0,25.0,Non essouché,0,10.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


split des datas

In [1504]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(data.drop(['fk_arb_etat'],axis = 1), data["fk_arb_etat"], test_size=0.2, random_state=42, stratify=data["fk_arb_etat"])
#, stratify=data["fk_arb_etat"]

In [1505]:
X_train

,longitude,latitude,haut_tot,haut_tronc,tronc_diam,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,remarquable,...,fk_nomtech_ROBPSE,fk_nomtech_SALBAB,fk_nomtech_SORAUC,fk_nomtech_TAXDIS,fk_nomtech_TILCOR,fk_nomtech_TILTOM,villeca_CASQ,villeca_VILLE,feuillage_Conifère,feuillage_Feuillu
4325,3.302073,49.861238,9.0,3.0,90.0,0,40.0,2.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1241,3.290787,49.850916,17.0,3.0,114.0,0,50.0,10.0,0.0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
5662,3.310778,49.845185,22.0,5.0,195.0,0,50.0,10.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
5877,3.266519,49.838346,6.0,2.0,41.0,0,20.0,5.0,0.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4719,3.297317,49.862002,18.0,4.0,100.0,0,40.0,10.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5875,3.266706,49.838299,6.0,2.0,40.0,0,20.0,5.0,0.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2369,3.309045,49.854809,3.0,2.0,20.0,0,5.0,3.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2107,3.263818,49.836429,18.0,3.0,125.0,0,30.0,10.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3131,3.309510,49.833082,7.0,1.0,80.0,0,20.0,5.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [1506]:
y_train

4325    Non essouché
1241        Essouché
5662        Essouché
5877        Essouché
4719        Essouché
            ...     
5875        Essouché
2369        Essouché
2107        Essouché
3131        Essouché
149         Essouché
Name: fk_arb_etat, Length: 153, dtype: object

## SMOTE

In [1507]:
sm = SMOTE(random_state=42, sampling_strategy=0.4)
X_train_smoted, y_train_smoted = sm.fit_resample(X_train, y_train)
#X_train_smoted, y_train_smoted = X_train, y_train # pour voir sans SMOTE

In [1508]:
y_train_smoted

0      Non essouché
1          Essouché
2          Essouché
3          Essouché
4          Essouché
           ...     
178    Non essouché
179    Non essouché
180    Non essouché
181    Non essouché
182    Non essouché
Name: fk_arb_etat, Length: 183, dtype: object

## Label Selection

In [1509]:
# Train the Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_smoted, y_train_smoted)

RandomForestClassifier(random_state=42)

In [1510]:
importances = rf.feature_importances_
feature_names = X_train_smoted.columns

feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importance_df

,Feature,Importance
0,longitude,0.040946
1,latitude,0.053898
2,haut_tot,0.063204
3,haut_tronc,0.036857
4,tronc_diam,0.064776
...,...,...
157,fk_nomtech_TILTOM,0.000000
158,villeca_CASQ,0.031616
159,villeca_VILLE,0.036351
160,feuillage_Conifère,0.013581


In [1511]:
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
feature_importance_df

,Feature,Importance
4,tronc_diam,0.064776
2,haut_tot,0.063204
1,latitude,0.053898
6,age_estim,0.047848
31,clc_secteur_Ecole maternelle Ernest Lavisse,0.042228
...,...,...
57,clc_secteur_Rue Alexandre Dumas,0.000000
128,fk_nomtech_FRAORN,0.000000
114,fk_nomtech_ACEPLAcri,0.000000
117,fk_nomtech_ACESACwie,0.000000


In [1512]:
# Select top N features (example selecting top 20 features)
top_features = feature_importance_df['Feature'][:20].values
X_train_selected = X_train_smoted[top_features]
X_test_selected = X_test[top_features]
top_features

array(['tronc_diam', 'haut_tot', 'latitude', 'age_estim',
       'clc_secteur_Ecole maternelle Ernest Lavisse', 'longitude',
       'clc_secteur_Stade Paul Debrésie', 'haut_tronc', 'villeca_VILLE',
       'clc_quartier_Quartier Saint-Jean', 'villeca_CASQ',
       "clc_secteur_Parc d'Isle Jacques Braconnier", 'fk_prec_estim',
       'fk_situation_Isolé', 'feuillage_Feuillu', 'fk_pied_gazon',
       'fk_nomtech_PINNIGnig', 'fk_nomtech_BETPEN', 'fk_nomtech_JUGREG',
       "clc_quartier_Quartier du faubourg d'Isle"], dtype=object)

In [1513]:
X_train_selected.shape

(183, 20)

In [1514]:
# Train the Random Forest model with selected features
start_time = time.time()
rf_selected = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_selected.fit(X_train_selected, y_train_smoted)
end_time = time.time()
#y_test_predicted = rf_selected.predict(X_test_selected)

In [1515]:
y_train_smoted.shape

(183,)

In [1516]:
X_train_selected.shape

(183, 20)

In [1517]:
from sklearn.metrics import accuracy_score
#cvs = cross_val_predict(rf_selected, X_train_selected, y_train_smoted, cv = 3, n_jobs=-1)
predicted_train = rf_selected.predict(X_train_selected)
confusion_matrix(y_train_smoted, predicted_train, normalize="true")

array([[1., 0.],
       [0., 1.]])

Etonnament satisfaisant mais ok pourquoi pas

In [1518]:
y_test.shape

(39,)

In [1519]:
y_test_predicted = rf_selected.predict(X_test_selected)
confusion_matrix(y_test, y_test_predicted, normalize="true")

array([[0.97058824, 0.02941176],
       [0.4       , 0.6       ]])

La matrice de confusion des données de test n'est pas très bonne, le nombre de True Negative n'est que de 0.6. Cependant les True Positive sont excellents. Pour le moment le meilleur modèle, même s'il est loin d'être excellent.

In [1520]:
print(f'Accuracy after feature selection: {accuracy_score(y_test, y_test_predicted):.3f}')

Accuracy after feature selection: 0.923


In [1521]:
y_test_predicted.shape

(39,)

## Export du modele et ses parametres de préparation

In [1522]:
import pickle
import dill
"""
def data_preaparation(data):
  import pandas
  indexDrop = data[((data['fk_arb_etat'] == 'ABATTU') | (data['fk_arb_etat'] == 'EN PLACE') | (data['fk_arb_etat'] =='REMPLACÉ') | (data['fk_arb_etat'] == 'SUPPRIMÉ'))].index
  data.drop(indexDrop, inplace = True)

  from sklearn.preprocessing import OneHotEncoder
  ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')
  
  categorical_data_cols = ['clc_quartier', 'clc_secteur', 'fk_stadedev', 'fk_port', 'fk_pied', 'fk_situation',  'fk_nomtech', 'villeca', 'feuillage']
  boolean_cols = ['fk_revetement', 'remarquable']
  
  for col in categorical_data_cols:
    ohetransform = ohe.fit_transform(data[[col]])
    data = data.join(ohetransform)

  data.drop(categorical_data_cols, axis = 1, inplace = True) 
  
  for col in boolean_cols:
    new_col = []
    for cell in data[col]:
      new_col.append(1 if cell=="Oui" else 0)
    data[col] = new_col

  return data
"""
  
prep_data_dict ={
  "model" : rf_selected,
  "encoders" : dict_encoders,
  "cat_cols_dist" : categorical_data_cols,
  "bool_cols_list" : boolean_cols, 
  "top_features" : top_features
}

dict = {
  "model" : rf_selected,
  "prep_data_dict" : prep_data_dict
}
  

with open('rf_model_1.pkl', 'wb') as f:
  pickle.dump(dict, f, protocol=pickle.HIGHEST_PROTOCOL)
  

## grid search

In [1523]:
"""
# train
classifier = RandomForestClassifier(random_state=42, n_jobs=-1)
params = {"n_estimators": [50, 100, 150, 200, 250, 300]}
gscv = GridSearchCV(estimator=classifier, param_grid=params, scoring="accuracy", cv=5, n_jobs=-1)
classifier_with_grid_search = gscv.fit(X_train, y_train)  # le meilleur modele

print(classifier_with_grid_search.best_params_)
from sklearn.metrics import accuracy_score
cvs = cross_val_predict(gscv, X_smoted, y_smoted, cv = 3, n_jobs=-1)

best_n = classifier_with_grid_search.best_params_["n_estimators"]
best_n
"""
best_n = 100

### validation

In [1524]:
#confusion_matrix(y_smoted, cvs, normalize="true")

### tests

In [1525]:
"""from sklearn.metrics import accuracy_score

y_test_predicted = classifier_with_grid_search.predict(X_test)
accuracy_score(y_test, y_test_predicted)
"""

'from sklearn.metrics import accuracy_score\n\ny_test_predicted = classifier_with_grid_search.predict(X_test)\naccuracy_score(y_test, y_test_predicted)\n'

In [1526]:
"""y_pred = classifier_with_grid_search.predict(X_test)
confusion_matrix(y_test, y_pred, normalize="true")"""

'y_pred = classifier_with_grid_search.predict(X_test)\nconfusion_matrix(y_test, y_pred, normalize="true")'

In [1527]:
"""y_pred = classifier_with_grid_search.predict(X_test)
y_proba = classifier_with_grid_search.predict_proba(X_test)

# Afficher la matrice de confusion et le rapport de classification
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))"""

'y_pred = classifier_with_grid_search.predict(X_test)\ny_proba = classifier_with_grid_search.predict_proba(X_test)\n\n# Afficher la matrice de confusion et le rapport de classification\nprint(confusion_matrix(y_test, y_pred))\nprint(classification_report(y_test, y_pred))'

In [1528]:
"""results = X_test.copy()
results['Actual'] = y_test
results['Predicted'] = y_pred
results['Probability_False'] = y_proba[:, 0]
results['Probability_True'] = y_proba[:, 1]"""

"results = X_test.copy()\nresults['Actual'] = y_test\nresults['Predicted'] = y_pred\nresults['Probability_False'] = y_proba[:, 0]\nresults['Probability_True'] = y_proba[:, 1]"

In [1529]:
"""print(results.head(20))"""

'print(results.head(20))'